In [61]:
import os
from pprint import pprint

import requests
import json

import pandas as pd
import tensorflow as tf

In [2]:
train_dataset_url = "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"

train_dataset_fp = tf.keras.utils.get_file(fname=os.path.basename(train_dataset_url),
                                           origin=train_dataset_url, cache_subdir = r'C:\Users\Aniket\Documents\Aniket\learning-ML\data')

print("Local copy of the dataset file: {}".format(train_dataset_fp))

Local copy of the dataset file: C:\Users\Aniket\Documents\Aniket\learning-ML\data\iris_training.csv


In [3]:
column_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species']
train_df = pd.read_csv(train_dataset_fp, names = column_names, skiprows = [0])
train_df

,sepal_length,sepal_width,petal_length,petal_width,species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0
...,...,...,...,...,...
115,5.5,2.6,4.4,1.2,1
116,5.7,3.0,4.2,1.2,1
117,4.4,2.9,1.4,0.2,0
118,4.8,3.0,1.4,0.1,0


In [4]:
def show_dataset(ds):
    for element in dataset:
        pprint(element)
        print('-----')
        
def show_batch(dataset):
    for batch, label in dataset.take(1):
        print("{:20s}: {}".format('labels',label))
        for key, value in batch.items():
            print("{:20s}: {}".format(key,value.numpy()))

In [5]:
train_labels = train_df.pop('species')

In [6]:
dataset = tf.data.Dataset.from_tensor_slices((dict(train_df),train_labels))
dataset = dataset.batch(batch_size = 10, drop_remainder=False)
dataset = dataset.repeat(3)
dataset = dataset.shuffle(buffer_size = 10000, seed=777, reshuffle_each_iteration=None)
# show_dataset(dataset)
show_batch(dataset)

labels              : [0 1 0 2 2 0 1 1 1 2]
sepal_length        : [4.7 6.6 5.5 7.7 6.1 4.9 5.5 5.7 6.  6.4]
sepal_width         : [3.2 2.9 3.5 3.  3.  3.1 2.4 2.9 2.9 2.7]
petal_length        : [1.3 4.6 1.3 6.1 4.9 1.5 3.8 4.2 4.5 5.3]
petal_width         : [0.2 1.3 0.2 2.3 1.8 0.1 1.1 1.3 1.5 1.9]


In [7]:
def pack_features_vector(features, labels):
    """Pack the features into a single array."""
    features = tf.stack(list(features.values()), axis=1)
    return features, labels

In [8]:
features, labels = next(iter(dataset))
features

{'sepal_length': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([5.4, 6.1, 6.5, 5.6, 6.3, 4.9, 6.8, 5.7, 6. , 5. ])>,
 'sepal_width': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([3.7, 2.9, 2.8, 2.7, 3.4, 3.1, 2.8, 2.8, 2.7, 3.5])>,
 'petal_length': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([1.5, 4.7, 4.6, 4.2, 5.6, 1.5, 4.8, 4.5, 5.1, 1.3])>,
 'petal_width': <tf.Tensor: shape=(10,), dtype=float64, numpy=array([0.2, 1.4, 1.5, 1.3, 2.4, 0.1, 1.4, 1.3, 1.6, 0.3])>}

In [9]:
features.values()

dict_values([<tf.Tensor: shape=(10,), dtype=float64, numpy=array([5.4, 6.1, 6.5, 5.6, 6.3, 4.9, 6.8, 5.7, 6. , 5. ])>, <tf.Tensor: shape=(10,), dtype=float64, numpy=array([3.7, 2.9, 2.8, 2.7, 3.4, 3.1, 2.8, 2.8, 2.7, 3.5])>, <tf.Tensor: shape=(10,), dtype=float64, numpy=array([1.5, 4.7, 4.6, 4.2, 5.6, 1.5, 4.8, 4.5, 5.1, 1.3])>, <tf.Tensor: shape=(10,), dtype=float64, numpy=array([0.2, 1.4, 1.5, 1.3, 2.4, 0.1, 1.4, 1.3, 1.6, 0.3])>])

In [10]:
list(features.values())

[<tf.Tensor: shape=(10,), dtype=float64, numpy=array([5.4, 6.1, 6.5, 5.6, 6.3, 4.9, 6.8, 5.7, 6. , 5. ])>,
 <tf.Tensor: shape=(10,), dtype=float64, numpy=array([3.7, 2.9, 2.8, 2.7, 3.4, 3.1, 2.8, 2.8, 2.7, 3.5])>,
 <tf.Tensor: shape=(10,), dtype=float64, numpy=array([1.5, 4.7, 4.6, 4.2, 5.6, 1.5, 4.8, 4.5, 5.1, 1.3])>,
 <tf.Tensor: shape=(10,), dtype=float64, numpy=array([0.2, 1.4, 1.5, 1.3, 2.4, 0.1, 1.4, 1.3, 1.6, 0.3])>]

In [11]:
tf.stack(list(features.values()), axis=1)

<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[5.4, 3.7, 1.5, 0.2],
       [6.1, 2.9, 4.7, 1.4],
       [6.5, 2.8, 4.6, 1.5],
       [5.6, 2.7, 4.2, 1.3],
       [6.3, 3.4, 5.6, 2.4],
       [4.9, 3.1, 1.5, 0.1],
       [6.8, 2.8, 4.8, 1.4],
       [5.7, 2.8, 4.5, 1.3],
       [6. , 2.7, 5.1, 1.6],
       [5. , 3.5, 1.3, 0.3]])>

In [12]:
dataset = dataset.map(pack_features_vector)
# features, labels = next(iter(dataset))
# features
show_dataset(dataset)

(<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[4.7, 3.2, 1.3, 0.2],
       [6.6, 2.9, 4.6, 1.3],
       [5.5, 3.5, 1.3, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [6.1, 3. , 4.9, 1.8],
       [4.9, 3.1, 1.5, 0.1],
       [5.5, 2.4, 3.8, 1.1],
       [5.7, 2.9, 4.2, 1.3],
       [6. , 2.9, 4.5, 1.5],
       [6.4, 2.7, 5.3, 1.9]])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 0, 2, 2, 0, 1, 1, 1, 2], dtype=int64)>)
-----
(<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[4.7, 3.2, 1.3, 0.2],
       [6.6, 2.9, 4.6, 1.3],
       [5.5, 3.5, 1.3, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [6.1, 3. , 4.9, 1.8],
       [4.9, 3.1, 1.5, 0.1],
       [5.5, 2.4, 3.8, 1.1],
       [5.7, 2.9, 4.2, 1.3],
       [6. , 2.9, 4.5, 1.5],
       [6.4, 2.7, 5.3, 1.9]])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 0, 2, 2, 0, 1, 1, 1, 2], dtype=int64)>)
-----
(<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[5. , 3.3, 1.4, 0.2],
       [6.6, 3. , 4.4, 1.4]

       [5.1, 3.8, 1.6, 0.2]])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([2, 1, 0, 0, 2, 0, 0, 2, 1, 0], dtype=int64)>)
-----


In [13]:
# alternate
dataset = tf.data.Dataset.from_tensor_slices((train_df.values,train_labels.values))
dataset = dataset.batch(batch_size = 10, drop_remainder=False)
dataset = dataset.repeat(3)
dataset = dataset.shuffle(buffer_size = 10000, seed=777, reshuffle_each_iteration=None)
show_dataset(dataset)
# show_batch(dataset)

(<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[4.7, 3.2, 1.3, 0.2],
       [6.6, 2.9, 4.6, 1.3],
       [5.5, 3.5, 1.3, 0.2],
       [7.7, 3. , 6.1, 2.3],
       [6.1, 3. , 4.9, 1.8],
       [4.9, 3.1, 1.5, 0.1],
       [5.5, 2.4, 3.8, 1.1],
       [5.7, 2.9, 4.2, 1.3],
       [6. , 2.9, 4.5, 1.5],
       [6.4, 2.7, 5.3, 1.9]])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 0, 2, 2, 0, 1, 1, 1, 2], dtype=int64)>)
-----
(<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[5. , 3.3, 1.4, 0.2],
       [6.6, 3. , 4.4, 1.4],
       [6.1, 2.8, 4. , 1.3],
       [5. , 3.2, 1.2, 0.2],
       [7. , 3.2, 4.7, 1.4],
       [6. , 3. , 4.8, 1.8],
       [7.4, 2.8, 6.1, 1.9],
       [5.8, 2.7, 5.1, 1.9],
       [6.2, 3.4, 5.4, 2.3],
       [5. , 2. , 3.5, 1. ]])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 1, 0, 1, 2, 2, 2, 2, 1], dtype=int64)>)
-----
(<tf.Tensor: shape=(10, 4), dtype=float64, numpy=
array([[6.3, 2.7, 4.9, 1.8],
       [5.7, 2.8, 4.1, 1.3]

In [14]:
def create_model():
    input1 = tf.keras.Input(shape=(4,))
    hidden1 = tf.keras.layers.Dense(units = 10, activation='relu')(input1)
    hidden2 = tf.keras.layers.Dense(units = 10, activation='relu')(hidden1)
    output1 = tf.keras.layers.Dense(units = 3, activation='softmax')(hidden2)
    
    model = tf.keras.models.Model(inputs = input1, outputs = output1, name= "functional1")
    
    
#     model.compile(optimizer='adam',
#                   loss='sparse_categorical_crossentropy',
#                   metrics=['accuracy'])
    return model
model = create_model()

In [15]:
features, labels = next(iter(dataset))

In [16]:
predictions = model(features)
predictions

<tf.Tensor: shape=(10, 3), dtype=float32, numpy=
array([[0.45596004, 0.1861081 , 0.35793185],
       [0.5713511 , 0.05101022, 0.37763867],
       [0.5827023 , 0.0460651 , 0.3712326 ],
       [0.55991   , 0.06057602, 0.37951398],
       [0.56621426, 0.04232045, 0.39146534],
       [0.44332236, 0.20159997, 0.35507768],
       [0.5895297 , 0.04251504, 0.36795533],
       [0.56063503, 0.05799313, 0.38137183],
       [0.55842537, 0.05239609, 0.38917854],
       [0.44714007, 0.1942002 , 0.35865968]], dtype=float32)>

In [ ]:
model.fit(dataset, epochs = 10)

In [21]:
# Custom Training LOOP
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

In [27]:
epochs = 10
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(dataset):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            preds = model(x_batch_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_batch_train, preds)

        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # Update training metric.
        train_acc_metric.update_state(y_batch_train, preds)

        # Log every 200 batches.
        print("Training loss (for one batch) at step %d: %.4f"% (step, float(loss_value)), end = '\r')
    print("\nTraining loss at epoch %d: %.4f"% (epoch+1, float(loss_value)))
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))
    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()



Start of epoch 0
Training loss (for one batch) at step 35: 0.2065
Training loss at epoch 1: 0.2065
Training acc over epoch: 0.9778

Start of epoch 1
Training loss (for one batch) at step 35: 0.3860
Training loss at epoch 2: 0.3860
Training acc over epoch: 0.9806

Start of epoch 2
Training loss (for one batch) at step 35: 0.1825
Training loss at epoch 3: 0.1825
Training acc over epoch: 0.9833

Start of epoch 3
Training loss (for one batch) at step 35: 0.1760
Training loss at epoch 4: 0.1760
Training acc over epoch: 0.9722

Start of epoch 4
Training loss (for one batch) at step 35: 0.0501
Training loss at epoch 5: 0.0501
Training acc over epoch: 0.9833

Start of epoch 5
Training loss (for one batch) at step 35: 0.3348
Training loss at epoch 6: 0.3348
Training acc over epoch: 0.9778

Start of epoch 6
Training loss (for one batch) at step 35: 0.2183
Training loss at epoch 7: 0.2183
Training acc over epoch: 0.9778

Start of epoch 7
Training loss (for one batch) at step 35: 0.1513
Training 

In [30]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        preds = model(x, training=True)
        loss_value = loss_fn(y, preds)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_acc_metric.update_state(y, preds)
    return loss_value

In [31]:
epochs = 10
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step, (x_batch_train, y_batch_train) in enumerate(dataset):
        loss_value = train_step(x_batch_train, y_batch_train)
        print("Training loss (for one batch) at step %d: %.4f"% (step, float(loss_value)), end = '\r')
    print("\nTraining loss at epoch %d: %.4f"% (epoch+1, float(loss_value)))
    # Display metrics at the end of each epoch.
    train_acc = train_acc_metric.result()
    print("Training acc over epoch: %.4f" % (float(train_acc),))
    # Reset training metrics at the end of each epoch
    train_acc_metric.reset_states()



Start of epoch 0
Training loss (for one batch) at step 35: 0.1385
Training loss at epoch 1: 0.1385
Training acc over epoch: 0.9806

Start of epoch 1
Training loss (for one batch) at step 35: 0.2090
Training loss at epoch 2: 0.2090
Training acc over epoch: 0.9722

Start of epoch 2
Training loss (for one batch) at step 35: 0.1088
Training loss at epoch 3: 0.1088
Training acc over epoch: 0.9833

Start of epoch 3
Training loss (for one batch) at step 35: 0.1946
Training loss at epoch 4: 0.1946
Training acc over epoch: 0.9750

Start of epoch 4
Training loss (for one batch) at step 35: 0.0512
Training loss at epoch 5: 0.0512
Training acc over epoch: 0.9806

Start of epoch 5
Training loss (for one batch) at step 35: 0.1221
Training loss at epoch 6: 0.1221
Training acc over epoch: 0.9750

Start of epoch 6
Training loss (for one batch) at step 35: 0.0934
Training loss at epoch 7: 0.0934
Training acc over epoch: 0.9806

Start of epoch 7
Training loss (for one batch) at step 35: 0.0965
Training 

In [41]:
class MyModel(tf.keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            preds = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss_value = self.compiled_loss(y, preds)

        # Compute gradients
        grads = tape.gradient(loss_value, self.trainable_variables)
        # Update weights
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, preds)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [42]:
input1 = tf.keras.Input(shape=(4,))
hidden1 = tf.keras.layers.Dense(units = 10, activation='relu')(input1)
hidden2 = tf.keras.layers.Dense(units = 10, activation='relu')(hidden1)
output1 = tf.keras.layers.Dense(units = 3, activation='softmax')(hidden2)

model = MyModel(inputs = input1, outputs = output1, name= "functional2")
model.compile(optimizer="adam", loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [43]:
model.fit(dataset, epochs=5)

Epoch 1/5
36/36 [==============================] - 0s 5ms/step - loss: 1.2408 - accuracy: 0.3500
Epoch 2/5
36/36 [==============================] - 0s 5ms/step - loss: 1.0316 - accuracy: 0.3500
Epoch 3/5
36/36 [==============================] - 0s 5ms/step - loss: 0.9741 - accuracy: 0.3500
Epoch 4/5
36/36 [==============================] - 0s 5ms/step - loss: 0.9282 - accuracy: 0.3500
Epoch 5/5
36/36 [==============================] - 0s 5ms/step - loss: 0.8789 - accuracy: 0.3750


In [74]:
# Custom Call Back
class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
#         print("End epoch {} of training; got log: {}".format(epoch, logs))
        if logs != None:
            logs['epoch'] = epoch
#         url = 'http://localhost:5000'
#         headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
#         requests.post(url, data = json.dumps(logs), headers=headers)
#         print(self.model)
        lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        print(lr)
        tf.keras.backend.set_value(self.model.optimizer.lr, lr * 0.9)
        
        
        

In [75]:
model.fit(dataset, epochs=5, callbacks = [CustomCallback()], verbose = 0)

0.0010000000474974513
0.0009000000427477062
0.0008100000559352338
0.0007290000794455409
0.0006561000482179224
